## 제주도 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = "chromedriver.exe"
driver = webdriver.Chrome(chromedriver)

In [3]:
insta_url = "https://www.instagram.com/"
driver.get(insta_url)
time.sleep(1)

In [4]:
with open("password.txt") as f:
    password = f.read()

In [5]:
email = "ra.iddo"
input_email = driver.find_element_by_css_selector("._2hvTZ.pexuQ.zyHYP")
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name("password")
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(3)

In [7]:
try:
    driver.find_element_by_css_selector(".sqdOP.yWX7d.y3zKF").click()
    time.sleep(5)
except:
    pass

In [8]:
try:
    driver.find_element_by_css_selector(".aOOlW.HoLwm").click()
    time.sleep(1)
except:
    pass

### 2. 제주도 맛집으로 검색

In [9]:
from urllib.parse import quote
keyword = "제주도맛집"
search_url = "https://www.instagram.com/explore/tags/"
url = f"{search_url}{quote(keyword)}"
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [10]:
driver.get(url)
time.sleep(5)

### 6. 첫번째 게시글 열기

In [11]:
driver.find_element_by_css_selector("div._9AhH0").click()

### 4. 게시글 정보 가져오기

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [13]:
import unicodedata
try:
    content = soup.select_one("div.C4VMK > span").text
    content = unicodedata.normalize("NFC",content)
except:
    content = " "

content

'(광고)제주도 최고의 갈치요리 맛집~!! 최고의 맛집이라 불리는 이유 드셔보시면 압니다~ 😍😍-갈치의 포근한 살과 꼬순맛을 제대로 맛볼수 있는 최고의 맛집!왜 최고의 맛집인지 한번 먹어봐요~-최상급의 해산물을 남김없이 들어간 최상급해물탕과!통갈치 해물찜을 딱 맛보면 술은 기본! 리얼 존맛!!-제주도에서 갈치요리, 해물요리는 무조건 갈치왕으로 가자-갈치왕 본점주소: 제주 서귀포시 일주서로 1146전화번호: 064-739-9333@kingfish1190예약 가능한지 사장님에게 문의하러 gogo~-#제주도맛집'

In [14]:
# 본문 내용에서 해쉬태그 가져오기, "#[^\s#,\\]+"
import re
tags = re.findall(r"#[^\s#,\\]+", content)
tags

['#제주도맛집']

In [16]:
# 3) 작성일자 정보 가져오기
date = soup.select_one("time.FH9sR.Nzb55")["datetime"][:10]
date

'2021-08-20'

In [17]:
# 4) 좋아요 수 가져오기 : 좋아요 1,296개
try:    
    like = soup.select_one("div.Nm9Fw").text[4:-1]
except:
    like = 0
like

0

In [18]:
# 5) 위치정보 가져오기
try:
    place = soup.select_one("div.M30cS")[0].text
    place = unicodedata.normalize("MFC",place)
except:
    place = ""
place

''

In [19]:
row = [content, date, like, place, tags]
row

['(광고)제주도 최고의 갈치요리 맛집~!! 최고의 맛집이라 불리는 이유 드셔보시면 압니다~ 😍😍-갈치의 포근한 살과 꼬순맛을 제대로 맛볼수 있는 최고의 맛집!왜 최고의 맛집인지 한번 먹어봐요~-최상급의 해산물을 남김없이 들어간 최상급해물탕과!통갈치 해물찜을 딱 맛보면 술은 기본! 리얼 존맛!!-제주도에서 갈치요리, 해물요리는 무조건 갈치왕으로 가자-갈치왕 본점주소: 제주 서귀포시 일주서로 1146전화번호: 064-739-9333@kingfish1190예약 가능한지 사장님에게 문의하러 gogo~-#제주도맛집',
 '2021-08-20',
 0,
 '',
 ['#제주도맛집']]

### 5. 다음 게시글로 이동

In [20]:
driver.find_element_by_css_selector("a._65Bje.coreSpriteRightPaginationArrow").click()
time.sleep(3)

In [134]:
driver.find_element_by_css_selector("button,wp06b")

<selenium.webdriver.remote.webelement.WebElement (session="e15f92df6f32c97a85eb156c35b44c67", element="3b3f73c5-19f2-4e42-8d4a-58cade767607")>

In [211]:
#6을 하기위해 팝업 화면을 없앰(즉 x 클릭)
driver.find_element_by_xpath("/html/body/div[6]/div[3]/button").click()

### 6. 여러개의 게시글 가져오기

In [239]:
from tqdm.notebook import tqdm

In [21]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]

In [22]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

#driver.find_element_by_xpath("/html/body/div[6]/div[3]/button").click()

In [232]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.to_csv("data/{keyword}.csv", index = False)

  0%|          | 0/500 [00:00<?, ?it/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a._65Bje.coreSpriteRightPaginationArrow"}
  (Session info: chrome=92.0.4515.159)


In [176]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

SyntaxError: invalid syntax (<ipython-input-176-3728c9d07a34>, line 1)

In [234]:
for keyword in ["제주도맛집","제주맛집","제주도관광","제주여행"]:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(10)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.to_csv("data/{keyword}.csv", index = False)

driver.find_element_by_xpath("/html/body/div[6]/div[3]/button").click()
time.sleep(7)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [275]:
driver.close()

In [237]:
df.head()

,content,date,like,place,tags
